# Setup

In [14]:
import wandb
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

# Data

In [22]:
# Create a nonlinear long-distance dataset
T = 20
D = 1
X = []
Y = []

def get_label(x, i1, i2, i3):
    if x[i1] < 0 and x[i2] < 0 and x[i3] < 0:
        return 1
    elif x[i1] < 0 and x[i2] > 0 and x[i3] > 0:
        return 1
    elif x[i1] > 0 and x[i2] < 0 and x[i3] > 0:
        return 1
    elif x[i1] > 0 and x[i2] > 0 and x[i3] < 0:
        return 1
    else:
        return 0
    
for t in range(5000):
    x = np.random.randn(T)
    X.append(x)
    y = get_label(x, 0, 1, 2)
    Y.append(y)

X = np.array(X)
Y = np.array(Y)
N = len(X)

# Model

In [23]:
# Configure the wandb sweep
sweep_config = {
    'method': 'bayes',
    'metric': {
      'name': 'val_accuracy',
      'goal': 'maximize'
    },
    'parameters': {
        'learning_rate': {
            'values': [1, 0.1, 0.01, 0.001, 0.0001]
        },
        'epochs': {
            'values': [80, 100, 200, 400]
        },
        'batch_size': {
            'values': [8, 16, 32, 64]
        },
        'optimizer': {
            'values': ['adam', 'nadam', 'sgd', 'rmsprop']
        },
        'activation_func': {
            'values': [None, 'tanh', 'relu', 'sigmoid']
        },
        'model': {
            'values': ['SimpleRNN', 'LSTM', 'GRU']
        },
    }
}

In [24]:
# Initialise sweep
sweep_id = wandb.sweep(sweep_config, project='tensorflow-test', entity='kavp')

Create sweep with ID: tsmolat6
Sweep URL: https://wandb.ai/kavp/tensorflow-test/sweeps/tsmolat6


In [25]:
# Mega function to define and train model and log results (used by the sweep)
def sweep_func():
    # Default hyperparameter values
    config_defaults = {
        'learning_rate': 0.001,
        'epochs': 200,
        'batch_size': 16,
        'optimizer': 'adam',
        'eager_mode': False,
        'activation_func': None,
        'model': 'LSTM',
    }

    # Initialise run
    wandb.init(config=config_defaults)

    # Variable to hold the sweep values
    config = wandb.config
    
    if config['eager_mode'] == True:
        tf.compat.v1.enable_eager_execution()
    elif config['eager_mode'] == False:
        tf.compat.v1.disable_eager_execution()
    else:
        raise ValueError('eager_mode property of wandb config could not be determined.') 

    # Create model
    inputs = np.expand_dims(X, -1)
    inp = tf.keras.layers.Input(shape=(T, D))
    if config['model'] == 'SimpleRNN':
        x = tf.keras.layers.SimpleRNN(5)(inp)
    elif config['model'] == 'LSTM':
        x = tf.keras.layers.LSTM(5)(inp)
    elif config['model'] == 'GRU':
        x = tf.keras.layers.GRU(5)(inp)
    else:
        raise ValueError('model property of wandb config could not be determined.')
    x = tf.keras.layers.Dense(1, activation=config['activation_func'])(x)

    model = tf.keras.models.Model(inp, x)

    model.compile(
        optimizer=config['optimizer'],
        loss='binary_crossentropy',
        metrics=['accuracy'],
        run_eagerly=config['eager_mode'],
    )

    with tf.compat.v1.Session() as sess:
        r = model.fit(inputs, Y, validation_split=0.5, epochs=config['epochs'], batch_size=config['batch_size'])
        wandb.tensorflow.log(tf.compat.v1.summary.merge_all())
        wandb.log({'loss': r.history['loss'][-1], 'val_loss': r.history['val_loss'][-1], 'accuracy': r.history['accuracy'][-1], 'val_accuracy': r.history['val_accuracy'][-1]})

        wandb_data = [[x,y] for (x,y) in zip(np.arange(0, config['epochs'], 1), r.history['loss'])]
        table = wandb.Table(data=wandb_data, columns = ["epoch", "loss"])
        wandb.log({"loss_against_epochs" : wandb.plot.line(table, "epoch", "loss", title="Training loss")})

        wandb_data = [[x,y] for (x,y) in zip(np.arange(0, config['epochs'], 1), r.history['val_loss'])]
        table = wandb.Table(data=wandb_data, columns = ["epoch", "val_loss"])
        wandb.log({"val_loss_against_epochs" : wandb.plot.line(table, "epoch", "val_loss", title="Validation loss")})

        wandb_data = [[x,y] for (x,y) in zip(np.arange(0, config['epochs'], 1), r.history['accuracy'])]
        table = wandb.Table(data=wandb_data, columns = ["epoch", "accuracy"])
        wandb.log({"accuracy_against_epochs" : wandb.plot.line(table, "epoch", "accuracy", title="Training accuracy")})

        wandb_data = [[x,y] for (x,y) in zip(np.arange(0, config['epochs'], 1), r.history['val_accuracy'])]
        table = wandb.Table(data=wandb_data, columns = ["epoch", "val_accuracy"])
        wandb.log({"val_accuracy_against_epochs" : wandb.plot.line(table, "epoch", "val_accuracy", title="Validation accuracy")})

        wandb.log_artifact(model)

        # Save model
        model.save(os.path.join(wandb.run.dir, 'model.h5'))

In [26]:
wandb.agent(sweep_id, sweep_func)

wandb: Agent Starting Run: ziqfk2my with config:
wandb: 	activation_func: tanh
wandb: 	batch_size: 8
wandb: 	epochs: 400
wandb: 	learning_rate: 0.01
wandb: 	model: LSTM
wandb: 	optimizer: sgd
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Train on 2500 samples, validate on 2500 samples
Epoch 1/400
2500/2500 [==============================] - 1s 388us/sample - loss: 0.7369 - accuracy: 0.5024 - val_loss: 0.7037 - val_accuracy: 0.4916
Epoch 2/400
2500/2500 [==============================] - 1s 317us/sample - loss: 0.6982 - accuracy: 0.5000 - val_loss: 0.6978 - val_accuracy: 0.4896
Epoch 3/400
2500/2500 [==============================] - 1s 316us/sample - loss: 0.6949 - accuracy: 0.5244 - val_loss: 0.6960 - val_accuracy: 0.4920
Epoch 4/400
2500/2500 [==============================] - 1s 330us/sample - loss: 0.6956 - accuracy: 0.5084 - val_loss: 0.6957 - val_accuracy: 0.4876
Epoch 5/400
2500/2500 [==============================] - 1s 318us/sample - loss: 0.6953 - accuracy: 0.5136 - val_loss: 0.6951 - val_accuracy: 0.4908
Epoch 6/400
2500/2500 [==============================] - 1s 324us/sample - loss: 0.6949 - accuracy: 0.4996 - val_loss: 0.6985 - val_accuracy: 0.4868
Epoch 7/400
2500/2500 [==============================] - 1

accuracy,▁
loss,▁
val_accuracy,▁
val_loss,▁
accuracy,0.5028
loss,0.6933
val_accuracy,0.4852
val_loss,0.69415


wandb: Agent Starting Run: zjxh0exl with config:
wandb: 	activation_func: relu
wandb: 	batch_size: 64
wandb: 	epochs: 200
wandb: 	learning_rate: 0.001
wandb: 	model: SimpleRNN
wandb: 	optimizer: adam
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Train on 2500 samples, validate on 2500 samples
Epoch 1/200
2500/2500 [==============================] - 0s 83us/sample - loss: 4.1565 - accuracy: 0.5032 - val_loss: 4.0615 - val_accuracy: 0.5036
Epoch 2/200
2500/2500 [==============================] - 0s 28us/sample - loss: 3.9255 - accuracy: 0.5044 - val_loss: 3.9463 - val_accuracy: 0.5040
Epoch 3/200
2500/2500 [==============================] - 0s 27us/sample - loss: 3.7598 - accuracy: 0.5008 - val_loss: 3.7166 - val_accuracy: 0.5020
Epoch 4/200
2500/2500 [==============================] - 0s 27us/sample - loss: 3.6174 - accuracy: 0.5000 - val_loss: 3.6446 - val_accuracy: 0.4996
Epoch 5/200
2500/2500 [==============================] - 0s 28us/sample - loss: 3.5121 - accuracy: 0.5016 - val_loss: 3.5253 - val_accuracy: 0.4956
Epoch 6/200
2500/2500 [==============================] - 0s 28us/sample - loss: 3.3890 - accuracy: 0.5008 - val_loss: 3.4501 - val_accuracy: 0.4948
Epoch 7/200
2500/2500 [==============================] - 0s 27us

...

accuracy,▁
loss,▁
val_accuracy,▁
val_loss,▁
accuracy,0.512
loss,0.69315
val_accuracy,0.4888
val_loss,0.69367


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: nbysk9ti with config:
wandb: 	activation_func: None
wandb: 	batch_size: 32
wandb: 	epochs: 400
wandb: 	learning_rate: 0.0001
wandb: 	model: LSTM
wandb: 	optimizer: nadam
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Train on 2500 samples, validate on 2500 samples
Epoch 1/400
2500/2500 [==============================] - 0s 169us/sample - loss: 3.0716 - accuracy: 0.5016 - val_loss: 1.7708 - val_accuracy: 0.5088
Epoch 2/400
2500/2500 [==============================] - 0s 82us/sample - loss: 1.4235 - accuracy: 0.5016 - val_loss: 1.1840 - val_accuracy: 0.5088
Epoch 3/400
2500/2500 [==============================] - 0s 81us/sample - loss: 1.1304 - accuracy: 0.5016 - val_loss: 1.0652 - val_accuracy: 0.5088
Epoch 4/400
2500/2500 [==============================] - 0s 81us/sample - loss: 1.0378 - accuracy: 0.5016 - val_loss: 0.9910 - val_accuracy: 0.5088
Epoch 5/400
2500/2500 [==============================] - 0s 82us/sample - loss: 0.9715 - accuracy: 0.5016 - val_loss: 0.9324 - val_accuracy: 0.5088
Epoch 6/400
2500/2500 [==============================] - 0s 85us/sample - loss: 0.9174 - accuracy: 0.5016 - val_loss: 0.8840 - val_accuracy: 0.5088
Epoch 7/400
2500/2500 [==============================] - 0s 84u

accuracy,▁
loss,▁
val_accuracy,▁
val_loss,▁
accuracy,0.6048
loss,0.65975
val_accuracy,0.5872
val_loss,0.67632


wandb: Ctrl + C detected. Stopping sweep.


In [27]:
wandb.finish()